<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [233]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import plotly.express as px


In [132]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
# Данные указаны в файле readme
DBNAME = *
USER = *
PASSWORD = *
HOST = *
PORT = *

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [165]:
# текст запроса
query_3_1 = f'''Select count(id)
                FROM  vacancies v
             '''


In [166]:
df=pd.read_sql_query(query_3_1,connection)
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\1585057861.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_3_1,connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [167]:
# # текст запроса
query_3_2 = f'''Select count(id)
                FROM employers e
             '''

In [168]:
df=pd.read_sql_query(query_3_2,connection)
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\1394967678.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_3_2,connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [169]:
query_3_3 = f'''Select count(id)
                FROM areas a
             '''   

In [170]:
df=pd.read_sql_query(query_3_3,connection)
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\3373722176.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_3_3,connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [171]:
query_3_4 = f'''Select count(id)
                FROM industries i
             '''

In [173]:
df=pd.read_sql_query(query_3_4,connection)
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\3429688010.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_3_4,connection)


,count
0,294


# выводы по предварительному анализу данных
Вывод:на 1362 региона приходится 23501 работодателей, которые предлагают 49197 вакансий в 294 сферах

***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [174]:
query_4_1 = f'''Select a.name area,
                       --подстчет количества вакансий по id 
                       count(v.id) cnt
                FROM vacancies v
                --присоеденяем  таблицу регионов для получения названия регионов для таблицы с вакансиями
                LEFT JOIN areas a on a.id=v.area_id
                --группируем по названию региона
                GROUP BY a.name
                --Сортируем по 2 колонке(кол-во вакансий) в порядке убывания
                ORDER BY 2 DESC
             '''

In [175]:
df=pd.read_sql_query(query_4_1,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\1093514247.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_4_1,connection);


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [176]:
query_4_2 = f'''Select count(v.id)
                FROM vacancies v
                --отбираем вакансии где заполнено хотя бы одно из двух полей с зарплатой
                WHERE salary_from is not null 
                   OR salary_to is not NULL
            '''

In [177]:
df=pd.read_sql_query(query_4_2,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\824439802.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_4_2,connection);


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [178]:
query_4_3 = f'''Select 
                    --среднее значение нижней границы зарплатной вилки округленное до целого
                     round(avg(v.salary_from)) as Минимальная,
                     --среднее значение верхней границы зарплатной вилки округленное до целого
                     round(avg(v.salary_to)) as Максимальная
                FROM vacancies v
              '''

In [179]:
df=pd.read_sql_query(query_4_3,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\1559611417.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_4_3,connection);


,Минимальная,Максимальная
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [180]:
query_4_4 = f'''Select 
                      count(id) Количесто,
                      schedule Тип_рабочего_графика,
                      employment Тип_трудоустройства
                FROM vacancies v
                --группировка по графику, а внутри по типу занятости
                GROUP BY schedule,employment
                --Сортируем по убыванию количества
                ORDER BY 1 DESC
             '''

In [181]:
df=pd.read_sql_query(query_4_4,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\1089769181.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_4_4,connection);


,Количесто,Тип_рабочего_графика,Тип_трудоустройства
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


In [237]:
res = df.groupby(['schedule','employment'],as_index=False).agg({'count':'mean'}).sort_values('count',ascending=False).iloc[:5]
display(res)
fig = px.bar(data_frame=res,
            x='schedule',
            y= 'count',
            height = 500,
            width= 1000,
            color='employment',
            template='plotly_dark',
            title= 'количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства',
            barmode= 'group'
           )
fig.update_layout(xaxis_title = 'ГРАФИК',
                 yaxis_title = 'КОЛ=-ВО',
                 title = {'font': dict(size = 15),
                          'x': 0.5,})
fig.update_traces(hovertemplate =  'График %{x} <br> Кол-во %{y}')
fig.show()
fig.write_html("Графики/Кол-во вакансий для каждого сочетания типа раб графика.html")

,schedule,employment,count
6,Полный день,Полная занятость,35367.0
14,Удаленная работа,Полная занятость,7802.0
2,Гибкий график,Полная занятость,1593.0
17,Удаленная работа,Частичная занятость,1312.0
10,Сменный график,Полная занятость,940.0


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [182]:
query_4_5 = f'''Select count(id) as Количество,experience as Требуемый_опыт
                FROM vacancies v
                --группируем по Требуемому опяту работы
                GROUP BY experience
                --Сортируем по 1 столбцу (кол-во вакансий)
                ORDER BY 1 
             '''

In [183]:
df=pd.read_sql_query(query_4_5,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\1365622505.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_4_5,connection);


,Количество,Требуемый_опыт
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


# выводы по детальному анализу вакансий
Вывод:распределение вакансий по регионам,  прямо коррелирует с количеством населения.Гаибольшее количество вакансий в Москве и Сант-Петербурге.
Средние значения для нижней и верхней границы зарплатной вилки составляют 71 и 110 тыс. рублей, соответственно
Проведя анализ мы видим,что только половина работодателей указало нижнюю и верхнюю границу заработной платы.
Больше всего требуемых вакансий с полным рабочим днем  и не удаленная работа с опытом работы от 1 года до 3-х лет
На  графике видно что максимальное кол-во вакансий приходится на сочетание Полного рабочего дня и Полной занятости.

***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [184]:
query_5_1 = f'''Select e.name ,count(v.id) 
                FROM employers e
                --присоеденияем таблицу с вакансиями
                LEFT JOIN vacancies v on e.id=v.employer_id
                --группируем по названию
                GROUP BY e.name
                --сортируем по кол-ву вакансий
                ORDER BY 2 desc
                --Отбираем первые 5 строк
                LIMIT 5
             '''

In [185]:
df=pd.read_sql_query(query_5_1,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\4065604187.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_5_1,connection);


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [186]:

query_5_2 = f'''
--Для выполнения задания в начале создадим 2 временные таблицы(СTE) для регионов и кол-ва вакансий в них и для регионов и кол-м роботодателей в них
                with vacant_count as(SELECT a.id id, COUNT(v.id) cnt_vacancies
                                     FROM vacancies v 
                                     LEFT JOIN areas a ON v.area_id = a.id
                                     --группируем по регионам
                                     GROUP BY a.id ),
                --и для регионов и кол-м роботодателей в них
                     employn_count as( SELECT a.id id ,COUNT(e.id) cnt_employers
                                       FROM employers e 
                                       LEFT JOIN areas a ON e.area = a.id
                                       --группируем по регионам
                                       GROUP BY a.id )
                Select a.name,
                       v.cnt_vacancies,
                       e.cnt_employers
                FROM areas a
                --присоеденяем временные таблицы
                LEFT JOIN vacant_count v  ON a.id = v.id
                LEFT  JOIN employn_count e ON a.id = e.id
                --выбираем регионы где кол-во работодателей не равно 0
                WHERE cnt_employers is  NOT NULL
                --Сортируем по кол-ву Вакансий , а после этой сортировки сортируем еще по кол-ву работодателей
                ORDER BY cnt_vacancies DESC ,cnt_employers DESC

'''

In [187]:
df=pd.read_sql_query(query_5_2,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\4267826828.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_5_2,connection);


,name,cnt_vacancies,cnt_employers
0,Россия,NaN,410
1,Казахстан,NaN,207
2,Московская область,NaN,75
3,Краснодарский край,NaN,19
4,Ростовская область,NaN,18
...,...,...,...
649,Гаврилов-Ям,1.0,1
650,Навои,1.0,1
651,Приморск (Ленинградская область),1.0,1
652,Баймак,1.0,1


3.Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.

Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.

In [188]:
query_5_3 = f''' Select e.name,count(distinct v.area_id) as cnt_area
                 FROM employers e
                 --присоеденяем таблицу с вакансиями
                 LEFT JOIN vacancies v on e.id=v.employer_id
                 -- группируем по id работодателя
                 GROUP BY e.id
                 --Сортируем по количеству работодателей
                 ORDER BY 2 DESC
             '''

In [189]:
df=pd.read_sql_query(query_5_3,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\340987382.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_5_3,connection);


,name,cnt_area
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [190]:
query_5_4 = f'''Select count(*)
                FROM employers e
                --присоеденяем полностью таблицу со сферами деятельности, получаем записи (строки) для каждого работодателя со всеми 
                --сферами деятельности
                FULL JOIN employers_industries i on e.id=i.employer_id
                --отбираем работодателей у которых не указана сфера деятельности
                WHERE i.industry_id is NULL
             '''

In [191]:
df=pd.read_sql_query(query_5_4,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\1015167197.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_5_4,connection);


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [192]:
query_5_5 = f'''Select e.name
                FROM employers e
                --присоеденяем таблицу для получения названия и id работодателей
                LEFT JOIN  employers_industries i on e.id=i.employer_id
                --Группируем по названию в алфавитном порядке
                GROUP BY 1
                --Отбираем только тех работодателей, у которых 4 сферы деятельности
                HAVING count(i.industry_id)=4
                --Сортируем по названию
                ORDER BY 1
                --пропускаем первые две строки, так как нум нужна 3 строка
                OFFSET 2
                --Оставляем одну запись
                LIMIT 1
            '''

In [193]:
df=pd.read_sql_query(query_5_5,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\623289658.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_5_5,connection);


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [194]:
query_5_6 = f'''Select count(DISTINCT employer_id)
                FROM employers_industries i
                --присоеденяем таблицу со сферами деятельности
                LEFT JOIN industries i_n on i_n.id=i.industry_id
                --отбираем нужную сферу деятельности
                WHERE i_n.name ILIKE 'Разработка программного обеспечения'
             '''

In [195]:
df=pd.read_sql_query(query_5_6,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\4293449947.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_5_6,connection);


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
url  = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
headers = {'User-Agent': 'Mozilla' }
           

response = requests.get(url,headers=headers)
soup  = BeautifulSoup(response.text,'lxml')
content = soup.find_all('td',style = 'text-align:center')
cities  = []
for x in content:
    res = x.previous_element.find('a').text.strip()
    cities.append(res)
print(cities)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [197]:
query_5_7 = f'''--Создадим временную таблицу для яндекса с регионами из заданного списка (города-миллионники) и количеством вакансий 
                With yandex as(
                               SELECT a.name,COUNT(*) cnt
                               FROM vacancies v
                               --в таблицу вакансий добавляем названия работодателей и регионов
                               LEFT JOIN employers e ON v.employer_id = e.id
                               LEFT JOIN areas a ON v.area_id = a.id 
                               --условие: только Яндекс и для городов миллионников
                               WHERE e.name iLIKE 'Яндекс' and  a.name IN {tuple(cities)}
                               --группируем
                               GROUP BY a.id
                              )
                SELECT *
                FROM yandex
                --добавляем к имеющейся таблице  общее количество вакансий по всем городам-миллионникам из нее же 
                UNION
                SELECT 'Total', SUM(cnt)
                FROM yandex
                --сортируем по кол-ву
                ORDER BY cnt 
            '''

In [198]:
df=pd.read_sql_query(query_5_7,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\2655730103.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_5_7,connection);


,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


# выводы по анализу работодателей
Вывод:
Россия лидер по кол-ву работодателей у которых нет вакансий
Работодатели на первом и пятом месте по количеству вакансий: Яндекс и Газпромнефть.
Если посмотреть количество регионов для каждого работодателя, то увидим чем больше ваканский у работодателя, 
тем больше число регионов,где он предлагает свои вакансии. 
У 8419 работодатедей не указана сфера деятельности, это достаточно много.
У 2ГИС указано 4 сферы деятельности.
У 3553 работодателей в качестве сферы деятельности указана "Разработка программного обеспечения".
Яндекс предоставляет 485 вакансий в 16 городах миллионников,что составляет четверть всех вакансий работодателя

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [199]:
query_6_1 = f'''SELECT count(*)
                FROM vacancies v
                --проверяем ,что в названии содержаться слова из условии,так как названия могут быть в любом регистре, 
                --приводем все к нижнему регисту
                WHERE  lower(v.name) ILIKE '%data%' OR lower(v.name) ILIKE'%данн%'
             '''

In [200]:
df=pd.read_sql_query(query_6_1,connection)
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\4017370364.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_6_1,connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [201]:
query_6_2 = f'''SELECT count(*)
                FROM vacancies v
                --отбираем вакансии по условиям указанные выше
                WHERE (v.name ILIKE '%data scientist%'
                   OR v.name ILIKE'%data science%'
                   OR v.name ILIKE'%исследователь данных%' 
                   OR  v.name ILIKE'%machine learning%' 
                   OR v.name ILIKE'%машинн%обучен%' 
                   OR (v.name ILIKE'%ML%' and v.name not ILIKE '%HTML%'))
                   AND (v.name ILIKE '%junior%' or v.experience ILIKE '%Нет опыта%' OR v.employment ILIKE '%Стажировка%') 
             '''

In [202]:
# результат запроса
df=pd.read_sql_query(query_6_2,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\887616430.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_6_2,connection);


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

В следующих заданиях мы продолжим работать с вакансиями по указанному выше условию (уже не учитывая вакансии уровня Junior).

In [203]:
query_6_3 = f''' SELECT count(*)
                FROM vacancies v
                --отбираем вакансии по условиям указанные выше
                WHERE (v.name ILIKE '%data scientist%'
                   OR v.name ILIKE'%data science%'
                   OR v.name ILIKE'%исследователь данных%' 
                   OR  v.name ILIKE'%machine learning%' 
                   OR v.name ILIKE'%машинн%обучен%' 
                   OR (v.name ILIKE'%ML%' and v.name not ILIKE '%HTML%'))
                   AND (v.key_skills ILIKE '%SQL%' or v.key_skills ILIKE '%postgres%') 
             '''

In [204]:
df=pd.read_sql_query(query_6_3,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\4235656378.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_6_3,connection);


,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [205]:
query_6_4 = f'''SELECT count(*)
                --отбираем по ключевому навыку из условия по критериям указанным выше как и в прошлом задании 
                FROM vacancies v
                WHERE  v.key_skills ILIKE '%Python%'  AND (v.name ILIKE '%data scientist%' OR v.name ILIKE'%data science%'
                    OR v.name ILIKE'%исследователь данных%' 
                    OR  v.name ILIKE'%machine learning%' 
                    OR v.name ILIKE'%машинн%обучен%'
                    OR (v.name ILIKE '%ML%' and v.name not ILIKE '%HTML%'))
            '''

In [206]:
# результат запроса
df=pd.read_sql_query(query_6_4,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\678946149.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_6_4,connection);


,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [207]:
query_6_5 = f'''SELECT
                -- вычисляем количество символов разделитетелей, да + 1 (так как разделитетелей на 1 меньше чем разделяемых элементов)
                --и округляем до 2 знаков
                      round(avg (length(v.key_skills) - length(replace(v.key_skills, '\t', ''))+1),2)
                FROM vacancies v
                --отбираем по ключевому навыку из условия по критериям указанным выше как и в прошлом задании 
                WHERE v.name ILIKE '%data scientist%' 
                   OR v.name ILIKE'%data science%'
                   OR v.name ILIKE'%исследователь данных%' 
                   OR  v.name ILIKE'%machine learning%' 
                   OR v.name ILIKE'%машинн%обучен%'
                   OR (v.name LIKE'%ML%' and v.name not ILIKE '%HTML%')
            '''

In [208]:
df=pd.read_sql_query(query_6_5,connection);
df

C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\266554403.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_6_5,connection);


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
query_6_6 = f'''SELECT 
                      experience,
                      --вычисляем среднюю зарплату ,так как у нас не у всех работодателей.Если заполнены оба поля с зарплатой, 
                      --то берем среднеарифметическое значений минимальной и максимальной зарплаты. 
                      --Если заполнено только одно из полей, берем его значение с помощью функции coalesce
                      ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2))::int avg_salary
                      FROM vacancies v
                      --отбираем по  критериям указанным выше как и в прошлом задании 
                      WHERE (v.name ILIKE '%data scientist%'
                         OR v.name ILIKE'%data science%'
                         OR v.name ILIKE'%исследователь данных%'
                         OR  v.name ILIKE'%machine learning%' 
                         OR v.name ILIKE'%машинн%обучен%'
                         OR (v.name LIKE'%ML%' and v.name not ILIKE '%HTML%'))
                        AND experience = 'От 3 до 6 лет'
                        AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
                     GROUP BY experience
                     
            '''

In [ ]:
df=pd.read_sql_query(query_6_6,connection);
df


C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\2049319578.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query_6_6,connection);


,experience,avg_salary
0,От 3 до 6 лет,243115


***

In [ ]:
# выводы по предметному анализу
Вывод:1771 вакансий имеет отношение к данным.
51 вакансия для начинающих дата-сайнтистов.
229 вакансия DS где основной навык SQL и postgres
357 вакансия DS где основной навык Python, Python более востебован.
для Вакансии DS в среднем необходим 6 ключевых навыков, это не мало.
Наибольшую зарплату предлагают для DS с опытом работы от 3 до 6 лет.
Но это связано с тем,что для опыта от 6 лет ЗП скорее всего обсуждается с кандитатом индивидуально...

Посмотрим в каких регионах выставляются вакансии и какая средняя зп

In [226]:
query_7_1 = f''' 
    select  a.name,
        round(avg(coalesce(v.salary_from,salary_to,(v.salary_from + v.salary_to)/2)))
    from public.VACANCIES as v 
    join public.areas as a on v.area_id = a.id
    where v.salary_from is not null or v.salary_to is not null
    group by 1
    order by 2 desc
    limit 10 
    '''
df= pd.read_sql_query(query_7_1,connection )
df




C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\2088602959.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql_query(query_7_1,connection )


,name,round
0,Германия,348270.0
1,Болгария,307681.0
2,Испания,300000.0
3,Черногория,270324.0
4,Литва,263019.0
5,Польша,257156.0
6,Петропавловка (Республика Бурятия),250000.0
7,Кипр,209573.0
8,Армения,206208.0
9,Турция,204919.0


Вывод: вакансии и работодатели которые выставляют эти вакансии охватывают регионы России и за её пределами.Так же можем сделать вывод, что ЗП в среднем выше за рубежом нежели в регионах России.

посмотрим сколько профессий требуют знаний Python 

In [ ]:
query_7_2 = f'''SELECT
                count(v.id) , 'Python' as "навыки"
                FROM  vacancirs as v
                where v.key_skills is not null and v.key_skills ILIKE '%Python%'
                union all
                select 
                count(v.id) , 'другие'
                from vacancies as v
                where v.key_skills is not null and v.key_skills  not ILIKE '%Python%'
    '''


df  = pd.read_sql_query(query_7_2,connection)
df


C:\Users\isyr\AppData\Local\Temp\ipykernel_20100\3396294318.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df  = pd.read_sql_query(query_7_2,connection)


,count,навыки
0,3461,Python
1,37280,другие


Вывод: можно сказать о дастоточно популярном яз. програмирования как "Python", много вакансий которые требуют знания Python и их 3461 вакансия от общего числа.

In [ ]:
connection.close()

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования
Общее количество вакансий 49197, на 23501 работодателей , но только  14906 с вакансиями.
Распределение вакансий по регионам,  прямо коррелирует с количеством населения.
Чем больше ваканский у работодателя,тем больше число регионов,где он предлагает свои вакансии
У 8419 работодатедей не указана сфера деятельности, это достаточно много
Больше всего требуемых вакансий с полным рабочим днем  и не удаленная работа с опытом работы от 1 года до 3-х лет
В51 вакансии указан уровень Junior, это примерно 10% от всех дата-сайентистов.
SQL и postgres в качестве ключевого навыков указаны в 229 вакансии для дата-сайентистов. Python популярнее – присутствует в 357 вакансии
Среднее кол-во навыков для DS 6
Наибольшую зарплату предлагают для DS с опытом работы от 3 до 6 лет.
Но это связано с тем,что для опыта от 6 лет ЗП скорее всего обсуждается с кандитатом индивидуально...
Начинающему сотруднику готовы платить примерно 75 тыс руб, при опыте работы более года в два раза больше, около 140 тыс.

